In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ

environ{'COMMAND_MODE': 'unix2003',
        'CONDA_EXE': '/Users/s23326/Dev/espnet/tools/miniconda/envs/espnet/bin/conda',
        'CONDA_PYTHON_EXE': '/Users/s23326/Dev/espnet/tools/miniconda/envs/espnet/bin/python',
        'CONDA_SHLVL': '0',
        'ELECTRON_NO_ATTACH_CONSOLE': '1',
        'HOME': '/Users/s23326',
        'HOMEBREW_CELLAR': '/opt/homebrew/Cellar',
        'HOMEBREW_PREFIX': '/opt/homebrew',
        'HOMEBREW_REPOSITORY': '/opt/homebrew',
        'INFOPATH': '/opt/homebrew/share/info:',
        'LAB': '/Users/sinos/OneDrive/document/lab',
        'LC_CTYPE': 'UTF-8',
        'LOGNAME': 's23326',
        'MallocNanoZone': '0',
        'OLDPWD': '/Users/s23326',
        'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined',
        'PAPER': '/Users/sinos/OneDrive/document/lab/mypaper',
        'PATH': '/Users/s23326/Dev/DialogueMock/.venv/bin:/Users/s23326/Dev/espnet/tools/miniconda/envs/espnet/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/opt/homebrew/opt/unzip/bin:/usr/loc

In [3]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import numpy as np
from typing import List

load_dotenv()

class EmbeddingBridge:
    def __init__(self):
        self.openai_model = os.environ.get("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
        model_version = os.environ.get("AZURE_OPENAI_MODEL_VERSION")
        api_key = os.environ.get("AZURE_OPENAI_API_KEY")
        endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
        
        self.client = AzureOpenAI(
            api_key=api_key,
            api_version=model_version,
            azure_endpoint=endpoint,
        )

    def get_embedding(self, text: str) -> List[float]:
        """
        単一のテキストのembeddingを取得します
        """
        response = self.client.embeddings.create(
            model=self.openai_model,
            input=text
        )
        return response.data

    def get_embeddings(self, texts: List[str]) -> List[List[float]]:
        """
        複数のテキストのembeddingを取得します
        """
        response = self.client.embeddings.create(
            model=self.openai_model,
            input=texts
        )
        return [item.embedding for item in response.data]

    def calculate_similarity(self, text1: str, text2: str) -> float:
        """
        2つのテキスト間のコサイン類似度を計算します
        """
        embedding1 = self.get_embedding(text1)
        embedding2 = self.get_embedding(text2)
        
        return self._cosine_similarity(embedding1, embedding2)

    def _cosine_similarity(self, vec1: List[float], vec2: List[float]) -> float:
        """
        2つのベクトル間のコサイン類似度を計算します
        """
        vec1 = np.array(vec1)
        vec2 = np.array(vec2)
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [4]:
embedding_bridge = EmbeddingBridge()

In [21]:
texts = [
    "それでお願いします",
    "はい",
    "お願いします",
    "それで大丈夫です"
]
import time
start = time.perf_counter()
embeddings = embedding_bridge.get_embeddings(texts)
end = time.perf_counter() - start
print(f"elapsed time: {end} sec")

elapsed time: 0.27385545799916144 sec


In [18]:
len(embeddings)

4

In [25]:
start = time.perf_counter()
embeddings = embedding_bridge.get_embeddings(texts[0])
end = time.perf_counter() - start
print(f"elapsed time: {end} sec")

elapsed time: 0.23873066699889023 sec


In [26]:
from sklearn.cluster import KMeans
from collections import defaultdict
import time

class ReservationIntentClassifier:
    def __init__(self, embedding_bridge):
        self.embedding_bridge = embedding_bridge
        self.kmeans = None
        self.intent_mapping = {}
        
    def train(self, intent_samples: dict):
        """
        サンプル文からモデルを学習
        
        Args:
            intent_samples: {
                "confirm": [...],
                "deny": [...],
                "change": [...]
            }
        """
        all_embeddings = []
        all_intents = []
        
        for intent, samples in intent_samples.items():
            embeddings = self.embedding_bridge.get_embeddings(samples)
            all_embeddings.extend(embeddings)
            all_intents.extend([intent] * len(samples))
            
        n_intents = len(intent_samples)
        self.kmeans = KMeans(n_clusters=n_intents)
        cluster_labels = self.kmeans.fit_predict(all_embeddings)
        
        # クラスタとintentの対応関係を作成
        cluster_to_intent = defaultdict(list)
        for cluster_label, intent in zip(cluster_labels, all_intents):
            cluster_to_intent[cluster_label].append(intent)
        
        # 各クラスタの多数決でintentを決定
        for cluster_label, intents in cluster_to_intent.items():
            intent_counts = defaultdict(int)
            for intent in intents:
                intent_counts[intent] += 1
            majority_intent = max(intent_counts.items(), key=lambda x: x[1])[0]
            self.intent_mapping[cluster_label] = majority_intent

    def predict(self, text: str, threshold: float = 0.8) -> tuple:
        """予約関連の発話からintentを予測"""
        embedding = self.embedding_bridge.get_embedding(text)
        
        # 全クラスタ中心との類似度を計算
        similarities = [
            self._cosine_similarity(embedding, center)
            for center in self.kmeans.cluster_centers_
        ]
        
        # 最も類似度の高いクラスタを選択
        best_cluster = np.argmax(similarities)
        best_similarity = similarities[best_cluster]
        
        if best_similarity < threshold:
            return "unknown", best_similarity
            
        predicted_intent = self.intent_mapping[best_cluster]
        return predicted_intent, best_similarity
    
    def _cosine_similarity(self, vec1, vec2):
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [37]:
embedding_bridge = EmbeddingBridge()
classifier = ReservationIntentClassifier(embedding_bridge)

# 予約確認用のtraining phrases
intent_samples = {
    "confirm": [
        "はい",
        "確定"
        "はい、お願いします",
        "そうですね",
        "それで大丈夫です",
        "問題ありません",
        "確定でお願いします",
        "それでいいです",
        "その通りです",
        "その予約でお願いします"
    ],
    "deny": [
        "いいえ",
        "結構です",
        "やめておきます",
        "キャンセルします",
        "予約はしません",
        "やっぱりやめます",
        "違います",
        "キャンセルでお願いします",
        "取り消してください",
        "辞めます"
    ],
    "change": [
        "変更",
        "変更できますか",
        "変更したいです",
        "時間を変更したいです",
        #"人数を5名から3名に変更できますか",
        "日にちを変更したいです",
        #"15時じゃなくて17時にしたいです",
        "人数を変更したいのですが",
        "時間を後ろにずらしたいです",
        "別の日にしたいです",
        "人数を増やしたいです",
        "もう少し遅い時間にできますか",
        #"日付を来週にしたいです"
    ]
}

# モデルの学習
print("Training model...")
start = time.perf_counter()
classifier.train(intent_samples)
end = time.perf_counter()
print(f"Training time: {end - start:.2f} sec")

#クラスタとintentの対応関係を確認
print("\nCluster to Intent mapping:")
print(classifier.intent_mapping)

# # テスト用の発話例
# test_texts = [
#     "はい、その内容で問題ありません",
#     "いいえ、予約はやめておきます",
#     "人数を3名に変更できますか？",
#     "時間をもう少し遅くしたいです",
#     "その予約で確定でお願いします",
#     "キャンセルします",
#     "19時に変更できますか",
#     "すみません、わかりません"  # unknown intentのテスト
# ]

# print("\nPrediction results:")
# for text in test_texts:
#     intent, confidence = classifier.predict(text)
#     print(f"Text: {text}")
#     print(f"Predicted intent: {intent}")
#     print(f"Confidence: {confidence:.3f}")
#     print("-" * 50)

Training model...
Training time: 2.75 sec

Cluster to Intent mapping:
{0: 'deny', 1: 'confirm', 2: 'change'}


In [39]:
test_texts = [
    "はい、その内容で問題ありません",
    "いいえ、予約はやめておきます",
    "人数を3名に変更できますか",
    "時間をもう少し遅くしたいです",
    "その予約で確定でお願いします",
    "キャンセルします",
    "19時に変更できますか",
    "すみません、わかりません"  # unknown intentのテスト
]
for text in test_texts:
    intent, confidence = classifier.predict(text, threshold=0.5)
    print(f"Text: {text}")
    print(f"Predicted intent: {intent}")
    print(f"Confidence: {confidence:.3f}")
    print("-" * 50)

Text: はい、その内容で問題ありません
Predicted intent: deny
Confidence: 0.588
--------------------------------------------------
Text: いいえ、予約はやめておきます
Predicted intent: confirm
Confidence: 0.664
--------------------------------------------------
Text: 人数を3名に変更できますか
Predicted intent: change
Confidence: 0.630
--------------------------------------------------
Text: 時間をもう少し遅くしたいです
Predicted intent: change
Confidence: 0.641
--------------------------------------------------
Text: その予約で確定でお願いします
Predicted intent: confirm
Confidence: 0.824
--------------------------------------------------
Text: キャンセルします
Predicted intent: deny
Confidence: 0.655
--------------------------------------------------
Text: 19時に変更できますか
Predicted intent: change
Confidence: 0.654
--------------------------------------------------
Text: すみません、わかりません
Predicted intent: deny
Confidence: 0.502
--------------------------------------------------


In [49]:
from typing import Dict, List, Tuple
import time

class ReservationIntentClassifier:
    def __init__(self, embedding_bridge):
        self.embedding_bridge = embedding_bridge
        self.intent_embeddings: Dict[str, List[List[float]]] = {}
        self.intent_phrases: Dict[str, List[str]] = {}
        
    def train(self, intent_samples: dict):
        """
        各intentのサンプル文のembeddingを保存
        
        Args:
            intent_samples: {
                "confirm": [...],
                "deny": [...],
                "change": [...]
            }
        """
        for intent, samples in intent_samples.items():
            embeddings = self.embedding_bridge.get_embeddings(samples)
            self.intent_embeddings[intent] = embeddings
            self.intent_phrases[intent] = samples
    
    def predict(self, text: str, threshold: float = 0.8) -> Tuple[str, float]:
        """
        各intentのサンプル文との類似度を計算し、
        最も多く閾値を超えたintentを予測として返す
        """
        # 入力テキストのembeddingを取得
        text_embedding = self.embedding_bridge.get_embedding(text)
        
        # 各intentごとの類似度スコアを計算
        intent_scores = defaultdict(list)
        
        for intent, embeddings in self.intent_embeddings.items():
            # 各サンプル文との類似度を計算
            for i, embedding in enumerate(embeddings):
                similarity = self._cosine_similarity(text_embedding, embedding)
                intent_scores[intent].append({
                    'score': similarity,
                    'phrase': self.intent_phrases[intent][i]
                })
        
        # 閾値を超えたサンプル数をカウント
        intent_matches = defaultdict(int)
        intent_avg_scores = defaultdict(float)
        
        for intent, scores in intent_scores.items():
            # 閾値を超えたサンプル数　
            matches = sum(1 for item in scores if item['score'] >= threshold)
            intent_matches[intent] = matches
            
            # 平均スコアも計算
            avg_score = matches / len(scores)
            intent_avg_scores[intent] = avg_score
        
        # 最も多くマッチしたintentを選択
        max_matches = max(intent_matches.values())
        
        if max_matches == 0:
            return "unknown", 0.0
        best_intent = max(intent_matches, key=lambda x: intent_matches[x])        
        
        # 最大マッチ数が同じintentが複数ある場合は平均スコアで判断
        # best_intents = [
        #     intent for intent, matches in intent_matches.items() 
        #     if matches == max_matches
        # ]
        
        # if len(best_intents) > 1:
        #     best_intent = max(
        #         best_intents,
        #         key=lambda x: intent_avg_scores[x]
        #     )
        # else:
        #     best_intent = best_intents[0]
            
        return best_intent, intent_avg_scores[best_intent]
    
    def predict_with_details(self, text: str, threshold: float = 0.8) -> dict:
        """
        より詳細な予測情報を返す
        """
        text_embedding = self.embedding_bridge.get_embedding(text)
        
        # 各intentの詳細なスコアを計算
        intent_details = {}
        
        for intent, embeddings in self.intent_embeddings.items():
            matches = []
            for i, embedding in enumerate(embeddings):
                similarity = self._cosine_similarity(text_embedding, embedding)
                if similarity >= threshold:
                    matches.append({
                        'phrase': self.intent_phrases[intent][i],
                        'similarity': similarity
                    })
            
            intent_details[intent] = {
                'matching_phrases': sorted(matches, key=lambda x: x['similarity'], reverse=True),
                'average_similarity': np.mean([m['similarity'] for m in matches]) if matches else 0.0,
                'match_count': len(matches)
            }
        
        # 最終的な予測を取得
        predicted_intent, confidence = self.predict(text, threshold)
        
        return {
            'prediction': predicted_intent,
            'confidence': confidence,
            'details': intent_details
        }
    
    def _cosine_similarity(self, vec1, vec2):
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


In [70]:
embedding_bridge = EmbeddingBridge()
classifier = ReservationIntentClassifier(embedding_bridge)

# training phrases
intent_samples = {
    "confirm": [
        "はい",
        "予約完了",
        "はい、お願いします",
        "そうですね",
        "それで大丈夫です",
        "問題ありません",
        "確定でお願いします",
        "その内容で大丈夫です",
        "それでいいです",
        "その通りです",
        "その予約でお願いします",
    ],
    "deny": [
        "いいえ",
        "やめておきます",
        "キャンセルします",
        "予約はしません",
        "キャンセルでお願いします",
        "取り消してください",
        "予約キャンセルで"
    ],
    "change": [
        "時間を変更したいです",
        "人数を5名から3名に変更できますか",
        "日にちを変更したいです",
        "15時じゃなくて17時にしたいです",
        "人数を変更したいのですが",
        "時間を後ろにずらしたいです",
        "別の日にしたいです",
        "人数を増やしたいです",
        "もう少し遅い時間にできますか",
        "日付を来週にしたいです",
        "やっぱり19時で",
        "ちがいます 5人です",
        "やっぱり明後日で",
        "違う 5時"
    ]
}

# モデルの学習
print("Training model...")
start = time.perf_counter()
classifier.train(intent_samples)
end = time.perf_counter()
print(f"Training time: {end - start:.2f} sec")

Training model...
Training time: 2.88 sec


In [67]:
# テスト用の発話例
test_texts = [
    "はい、その内容で問題ありません",
    "いいえ、予約はやめておきます",
    "人数を3名に変更できますか？",
    "時間をもう少し遅くしたいです",
    "その予約で確定でお願いします",
    "キャンセルします",
    "19時に変更できますか",
    "すみません、わかりません"  # unknown intentのテスト
]
import time
print("\nBasic prediction results:")
for text in test_texts:
    tic = time.perf_counter()
    intent, score = classifier.predict(text, threshold=0.6)
    toc = time.perf_counter() - tic
    print(f"\nText: {text}")
    print(f"Predicted intent: {intent}")
    print(f"Confidence: {score:.3f}")
    print(f"Elapsed time: {toc:.2f} sec")


Basic prediction results:

Text: はい、その内容で問題ありません
Predicted intent: confirm
Confidence: 0.182
Elapsed time: 0.25 sec

Text: いいえ、予約はやめておきます
Predicted intent: deny
Confidence: 0.200
Elapsed time: 0.27 sec

Text: 人数を3名に変更できますか？
Predicted intent: change
Confidence: 0.143
Elapsed time: 0.27 sec

Text: 時間をもう少し遅くしたいです
Predicted intent: change
Confidence: 0.286
Elapsed time: 0.30 sec

Text: その予約で確定でお願いします
Predicted intent: confirm
Confidence: 0.273
Elapsed time: 0.26 sec

Text: キャンセルします
Predicted intent: deny
Confidence: 0.400
Elapsed time: 0.25 sec

Text: 19時に変更できますか
Predicted intent: change
Confidence: 0.071
Elapsed time: 0.26 sec

Text: すみません、わかりません
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.26 sec


In [71]:
# テスト用の発話例
test_texts = [
    "はい",
    "やっぱりなしで",
    "人数を3名に変更できますか？",
    "やっぱり時間をもう少し遅くしたいです",
    "やっぱり5人で",
    "やっぱり17時で",
    "違う16時です",
    "ちがう4人です",
    "違います 10人です",
    "その予約で確定でお願いします",
    "やっぱりキャンセルします",
    "19時に変更できますか",
    "すみませんわかりません"  # unknown intentのテスト
]
import time
print("\nBasic prediction results:")
for text in test_texts:
    tic = time.perf_counter()
    intent, score = classifier.predict(text, threshold=0.6)
    toc = time.perf_counter() - tic
    print(f"\nText: {text}")
    print(f"Predicted intent: {intent}")
    print(f"Confidence: {score:.3f}")
    print(f"Elapsed time: {toc:.2f} sec")


Basic prediction results:

Text: はい
Predicted intent: confirm
Confidence: 0.182
Elapsed time: 0.29 sec

Text: やっぱりなしで
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.26 sec

Text: 人数を3名に変更できますか？
Predicted intent: change
Confidence: 0.143
Elapsed time: 0.26 sec

Text: やっぱり時間をもう少し遅くしたいです
Predicted intent: change
Confidence: 0.286
Elapsed time: 0.26 sec

Text: やっぱり5人で
Predicted intent: change
Confidence: 0.071
Elapsed time: 0.31 sec

Text: やっぱり17時で
Predicted intent: change
Confidence: 0.143
Elapsed time: 0.28 sec

Text: 違う16時です
Predicted intent: change
Confidence: 0.071
Elapsed time: 0.26 sec

Text: ちがう4人です
Predicted intent: change
Confidence: 0.071
Elapsed time: 0.25 sec

Text: 違います 10人です
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.27 sec

Text: その予約で確定でお願いします
Predicted intent: confirm
Confidence: 0.273
Elapsed time: 0.27 sec

Text: やっぱりキャンセルします
Predicted intent: deny
Confidence: 0.429
Elapsed time: 0.25 sec

Text: 19時に変更できますか
Predicted intent: change
Confide

In [58]:
# 音声認識誤りを含むテストケース
asr_error_test_texts = [
    # 1. 区切り誤り
    "はいそうです",  # 「はい、そうです」
    "はいおねがいします",  # 「はい、お願いします」
    "いいえけっこうです",  # 「いいえ、結構です」
    
    # 2. 助詞の誤認識・欠落
    "人数は3名で変更できますか",  # 「を」→「は」
    "時間長くしたいです",  # 「を」の欠落
    "予約キャンセルします",  # 「を」の欠落
    
    # 3. 似た音の誤認識
    "じかんをへんこうしたいです",  # 「変更」→「へんこう」
    "にんずうを変えたいです",  # 「人数」→「にんずう」
    "はい、そうでず",  # 「です」→「でず」
    
    # 4. 文末の途切れ
    "はい、お願いし",  # 「お願いします」の途切れ
    "時間を変更したい",  # 「したいです」の途切れ
    "キャンセルで",  # 「キャンセルでお願いします」の途切れ
    
    # 5. フィラーや言い淀み
    "えーと、はい、そうです",
    "あの、時間をー、変更したいです",
    "うーんと、キャンセル、します",
    
    # 6. 音の挿入・重複
    "はいい、おねがいします",  # 「い」の重複
    "じかんんを変更したいです",  # 「ん」の重複
    "キャンセルルします",  # 「ル」の重複
    
    # 7. 背景ノイズや途切れによる部分的な認識
    "＊＊はい、お願いします",  # 冒頭の欠落
    "時間を変更＊＊たいです",  # 中間の欠落
    "キャンセルし＊＊",  # 末尾の欠落
    
    # 8. 方言や口語表現
    "へー、ええよ",  # 「はい」の口語
    "ちゃう、やっぱ変更して",  # 「違います」の口語
    "んー、そうやな",  # 「はい」の方言
    
    # 9. 複合的なエラー
    "えーと、じかんんをへんこうしたい＊＊",  # フィラー＋重複＋誤認識＋途切れ
    "あのー、にんずうかえたいんですけど",  # フィラー＋誤認識＋助詞欠落
    "はいい、そうでず＊＊",  # 重複＋誤認識＋途切れ
]

# テスト実行
print("\nASR Error Test Results:")
for text in asr_error_test_texts:
    tic = time.perf_counter()
    intent, score = classifier.predict(text, threshold=0.6)
    toc = time.perf_counter() - tic
    print(f"\nText: {text}")
    print(f"Predicted intent: {intent}")
    print(f"Confidence: {score:.3f}")
    print(f"Elapsed time: {toc:.2f} sec")


ASR Error Test Results:

Text: はいそうです
Predicted intent: confirm
Confidence: 0.273
Elapsed time: 0.82 sec

Text: はいおねがいします
Predicted intent: confirm
Confidence: 0.091
Elapsed time: 0.25 sec

Text: いいえけっこうです
Predicted intent: deny
Confidence: 0.182
Elapsed time: 0.25 sec

Text: 人数は3名で変更できますか
Predicted intent: change
Confidence: 0.200
Elapsed time: 0.26 sec

Text: 時間長くしたいです
Predicted intent: change
Confidence: 0.200
Elapsed time: 0.26 sec

Text: 予約キャンセルします
Predicted intent: deny
Confidence: 0.545
Elapsed time: 0.26 sec

Text: じかんをへんこうしたいです
Predicted intent: change
Confidence: 0.200
Elapsed time: 0.29 sec

Text: にんずうを変えたいです
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.31 sec

Text: はい、そうでず
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.27 sec

Text: はい、お願いし
Predicted intent: confirm
Confidence: 0.182
Elapsed time: 0.36 sec

Text: 時間を変更したい
Predicted intent: change
Confidence: 0.400
Elapsed time: 0.26 sec

Text: キャンセルで
Predicted intent: deny
Confidence: 0.273
Ela

In [60]:
# より実際的な音声認識誤りを含むテストケース
asr_error_test_texts = [
    # 1. 固有名詞の誤認識
    "4名でよやくおねがいします",  # 「予約」→「よやく」
    "とうきょうレストランでおねがいします",  # 「東京」→「とうきょう」
    
    # 2. 単語の部分変化
    "人数変蔵したいです",  # 「変更」→「変蔵」
    "予約完了しました",  # 「かんりょう」→「かんろう」
    "時間を告更したいです",  # 「変更」→「告更」
    
    # 3. 同音異義語
    "感激です、そのままおねがいします",  # 「確定」→「感激」
    "刊行をおねがいします",  # 「完了」→「刊行」
    "予約を完走します",  # 「完了」→「完走」
    
    # 4. ランダムな単語置換
    "ステップでおねがいします",  # 「はい」→「ステップ」
    "フォークです、キャンセルします",  # 「はい」→「フォーク」
    "テーブルを変更したいです",  # 「時間」→「テーブル」
    
    # 5. 分割ミス
    "はい今日精彩です",  # 「はい、確定です」
    "時間矯正したいです",  # 「時間変更したいです」
    "予約カレー了します",  # 「予約完了します」
    
    # 6. 無変換
    "じかん変更したいです",  # 「時間」→「じかん」
    "にんずう変更おねがいします",  # 「人数」→「にんずう」
    "よやく確定します",  # 「予約」→「よやく」
    
    # 7. その他
    "3名に変更に したいです",  # 余分な「に」の挿入
    "17時へ 30分です",  # 不自然な区切り
    "予約を2名むのお願いします",  # 不自然な助詞の挿入
    
    # 8. 複合的なエラー
    "時間矯正に感激したいです",  # 分割ミス + 同音異義語
    "にんずう変蔵カレーです",  # 無変換 + 部分変化 + 分割ミス
    "よやくステップ了します",  # 無変換 + ランダム置換 + 分割ミス
    
    # 9. 数字や時刻の誤認識
    "3名を5名に変更したいです",  # 「3」→「三」や「さん」
    "17時を18時半に変更したいです",  # 「17」→「じゅうなな」
    "予約を2名から3名に変更します",  # 数字の誤認識
]
print("\nASR Error Test Results:")
for text in asr_error_test_texts:
    tic = time.perf_counter()
    intent, score = classifier.predict(text, threshold=0.6)
    toc = time.perf_counter() - tic
    print(f"\nText: {text}")
    print(f"Predicted intent: {intent}")
    print(f"Confidence: {score:.3f}")
    print(f"Elapsed time: {toc:.2f} sec")


ASR Error Test Results:

Text: 4名でよやくおねがいします
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.78 sec

Text: とうきょうレストランでおねがいします
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.25 sec

Text: 人数変蔵したいです
Predicted intent: change
Confidence: 0.200
Elapsed time: 0.27 sec

Text: 予約完了しました
Predicted intent: confirm
Confidence: 0.182
Elapsed time: 0.28 sec

Text: 時間を告更したいです
Predicted intent: change
Confidence: 0.400
Elapsed time: 0.25 sec

Text: 感激です、そのままおねがいします
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.30 sec

Text: 刊行をおねがいします
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.27 sec

Text: 予約を完走します
Predicted intent: confirm
Confidence: 0.182
Elapsed time: 0.26 sec

Text: ステップでおねがいします
Predicted intent: unknown
Confidence: 0.000
Elapsed time: 0.25 sec

Text: フォークです、キャンセルします
Predicted intent: deny
Confidence: 0.200
Elapsed time: 0.28 sec

Text: テーブルを変更したいです
Predicted intent: change
Confidence: 0.100
Elapsed time: 0.25 sec

Text: はい今日精彩です
Predicted

In [53]:
from typing import List, Tuple
import pyopenjtalk
import Levenshtein
import jaconv
from functools import lru_cache

class TextSimilarityClassifier:
    def __init__(self, intent_samples: Dict[str, List[str]]):
        self.intent_samples = intent_samples

    def normalize_text(self, text: str) -> str:
        """テキストの正規化"""
        # カタカナとアルファベットをひらがなに変換
        text = jaconv.kata2hira(text.lower())
        # 空白と記号を削除
        text = ''.join(char for char in text if char.isalnum() or char.isspace())
        return text

    @lru_cache(maxsize=1024)
    def text_to_phonemes(self, text: str) -> str:
        """pyopenjtalkを使用して音素列に変換"""
        # pyopenjtalkで音素列を取得
        phonemes = pyopenjtalk.g2p(text)
        # 音素列から不要な記号を削除
        phonemes = phonemes.replace(' ', '').replace('+', '').replace('*', '')
        return phonemes
        
    def calculate_similarity(self, text1: str, text2: str) -> Tuple[float, float]:
        """文字レベルと音素レベルの類似度を計算"""
        # 文字レベルの類似度
        norm_text1 = self.normalize_text(text1)
        norm_text2 = self.normalize_text(text2)
        char_similarity = 1 - Levenshtein.distance(norm_text1, norm_text2) / max(len(norm_text1), len(norm_text2), 1)

        # 音素レベルの類似度
        phon_text1 = self.text_to_phonemes(text1)
        phon_text2 = self.text_to_phonemes(text2)
        phon_similarity = 1 - Levenshtein.distance(phon_text1, phon_text2) / max(len(phon_text1), len(phon_text2), 1)

        return char_similarity, phon_similarity
        

    def predict(self, text: str, char_threshold: float = 0.6, phon_threshold: float = 0.6) -> Tuple[str, float, float, Dict]:
        """最も類似度の高いintentを予測"""
        best_intent = "unknown"
        best_char_sim = 0.0
        best_phon_sim = 0.0
        similarities = {intent: [] for intent in self.intent_samples.keys()}
        
        # スコアを保持する辞書を初期化
        scores = {intent: {
            'char_count': 0,  # 文字類似度の閾値を超えた数
            'phon_count': 0,  # 音素類似度の閾値を超えた数
            'total_samples': len(self.intent_samples[intent]),  # サンプル総数
            'char_score': 0.0,  # 文字類似度スコア
            'phon_score': 0.0  # 音素類似度スコア
        } for intent in self.intent_samples.keys()}
        
        results = {
            'total_samples': len(self.intent_samples[intent]),
            'total_score': 0.0,
        }

        # 各intentのサンプルに対して類似度を計算
        for intent, samples in self.intent_samples.items():
            for sample in samples:
                char_sim, phon_sim = self.calculate_similarity(text, sample)
                similarities[intent].append({
                    'sample': sample,
                    'char_sim': char_sim,
                    'phon_sim': phon_sim
                })
                
                # 閾値を超えた場合にカウントを増やす
                if char_sim >= char_threshold:
                    scores[intent]['char_count'] += 1
                if phon_sim >= phon_threshold:
                    scores[intent]['phon_count'] += 1

        # スコアを計算
        for intent in scores:
            total_samples = scores[intent]['total_samples']
            scores[intent]['char_score'] = scores[intent]['char_count'] / total_samples
            scores[intent]['phon_score'] = scores[intent]['phon_count'] / total_samples
            
            # 最高スコアの更新
            current_total_score = scores[intent]['char_score'] + scores[intent]['phon_score']
            if current_total_score > best_char_sim + best_phon_sim:
                best_intent = intent
                best_char_sim = scores[intent]['char_score']
                best_phon_sim = scores[intent]['phon_score']
                
            results[intent]["total_score"] = current_total_score
            
        return best_intent, best_char_sim, best_phon_sim, results



In [61]:
from typing import Dict, List, Tuple
from functools import lru_cache

class TextSimilarityClassifier:
    def __init__(self, intent_samples: Dict[str, List[str]]):
        self.intent_samples = intent_samples

    def normalize_text(self, text: str) -> str:
        """テキストの正規化"""
        # カタカナとアルファベットをひらがなに変換
        text = jaconv.kata2hira(text.lower())
        # 空白と記号を削除
        text = ''.join(char for char in text if char.isalnum() or 'ぁ' <= char <= 'ん')
        return text

    @lru_cache(maxsize=1024)
    def text_to_phonemes(self, text: str) -> str:
        """pyopenjtalkを使用して音素列に変換"""
        try:
            # pyopenjtalkで音素列を取得
            _, phonemes = pyopenjtalk.g2p(text)
            # 音素列から不要な記号を削除
            phonemes = phonemes.replace(' ', '').replace('+', '').replace('*', '')
            return phonemes
        except:
            return self.normalize_text(text)

    def calculate_similarity(self, text1: str, text2: str) -> Tuple[float, float]:
        """文字レベルと音素レベルの類似度を計算"""
        # 文字レベルの類似度
        norm_text1 = self.normalize_text(text1)
        norm_text2 = self.normalize_text(text2)
        char_similarity = 1 - Levenshtein.distance(norm_text1, norm_text2) / max(len(norm_text1), len(norm_text2), 1)

        # 音素レベルの類似度
        phon_text1 = self.text_to_phonemes(text1)
        phon_text2 = self.text_to_phonemes(text2)
        phon_similarity = 1 - Levenshtein.distance(phon_text1, phon_text2) / max(len(phon_text1), len(phon_text2), 1)

        return char_similarity, phon_similarity

    def predict(self, text: str, char_threshold: float = 0.6, phon_threshold: float = 0.6) -> Tuple[str, float, float, Dict]:
        """最も類似度の高いintentを予測"""
        best_intent = "unknown"
        best_char_sim = 0.0
        best_phon_sim = 0.0
        
        # 結果を格納する辞書を初期化
        results = {intent: {
            'char_count': 0,
            'phon_count': 0,
            'total_samples': len(samples),
            'char_score': 0.0,
            'phon_score': 0.0,
            'total_score': 0.0
        } for intent, samples in self.intent_samples.items()}

        # 各intentのサンプルに対して類似度を計算
        for intent, samples in self.intent_samples.items():
            for sample in samples:
                char_sim, phon_sim = self.calculate_similarity(text, sample)
                
                # 閾値を超えた場合にカウントを増やす
                if char_sim >= char_threshold:
                    results[intent]['char_count'] += 1
                if phon_sim >= phon_threshold:
                    results[intent]['phon_count'] += 1

        # 各intentのスコアを計算
        for intent, result in results.items():
            total_samples = result['total_samples']
            if total_samples > 0:
                result['char_score'] = result['char_count'] / total_samples
                result['phon_score'] = result['phon_count'] / total_samples
                result['total_score'] =( result['char_score'] + result['phon_score']) / 2

                # 最高スコアの更新
                if result['total_score'] > best_char_sim + best_phon_sim:
                    best_intent = intent
                    best_char_sim = result['char_score']
                    best_phon_sim = result['phon_score']

        return best_intent, best_char_sim, best_phon_sim, results

In [69]:
intent_samples = {
    "confirm": [
        "はい",
        "予約完了",
        "はい、お願いします",
        "そうですね",
        "それで大丈夫です",
        "問題ありません",
        "確定でお願いします",
        "その内容で大丈夫です",
        "それでいいです",
        "その通りです",
        "その予約でお願いします",
    ],
    "cancel": [
        "いいえ",
        "やめておきます",
        "キャンセルします",
        "予約はしません",
        "キャンセルでお願いします",
        "取り消してください",
        "予約キャンセルで",
        "無しで",
    ],
    "change": [
        "時間を変更したいです",
        "人数を5名から3名に変更できますか",
        "日にちを変更したいです",
        "15時じゃなくて17時にしたいです",
        "人数を変更したいのですが",
        "時間を後ろにずらしたいです",
        "別の日にしたいです",
        "人数を増やしたいです",
        "もう少し遅い時間にできますか",
        "日付を来週にしたいです",
        "19時です",
        "3人です",
        "ちがいます 5人です",
        "やっぱり明後日で",
        "違う 5時"
    ]
}
classifier = TextSimilarityClassifier(intent_samples)

In [70]:
from pprint import pprint
# 使用例
def test_classifier(classifier, test_cases, details=False):
    for text in test_cases:
        intent, char_sim, phon_sim, results = classifier.predict(text)
        print(f"\nInput: {text}")
        print(f"Predicted Intent: {intent}")
        print(f"Best Character Similarity: {char_sim:.3f}")
        print(f"Best Phoneme Similarity: {phon_sim:.3f}")
        pprint(results)
        # if details:
        #     print("\nTop matches for each intent:")
        #     for intent, matches in results.items():
        #         # 類似度の合計でソート
        #         sorted_matches = sorted(matches, 
        #                             key=lambda x: x['char_sim'] + x['phon_sim'], 
        #                             reverse=True)[:3]
        #         print(f"\n{intent}:")
        #         for match in sorted_matches:
        #             print(f"  Sample: {match['sample']}")
        #             print(f"  - Char sim: {match['char_sim']:.3f}, Phon sim: {match['phon_sim']:.3f}")

test_texts = [
    "はい",
    "やっぱりなしで",
    "人数を3名に変更できますか？",
    "やっぱり時間をもう少し遅くしたいです",
    "やっぱり5人で",
    "やっぱり17時で",
    "違う16時です",
    "ちがう4人です",
    "違います 10人です",
    "その予約で確定でお願いします",
    "やっぱりキャンセルします",
    "19時に変更できますか",
    "すみませんやっぱり",
    "すみませんやっぱり明後日で",
    "すみませんやっぱり5人で",
    "すみませんやっぱり二人で",
    "すみませんわかりません",  # unknown intentのテスト
]

test_classifier(classifier, test_texts)

ValueError: not enough values to unpack (expected 4, got 3)

In [77]:
from typing import Dict, List, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
from functools import lru_cache

class TextSimilarityClassifier:
    def __init__(self, intent_samples: Dict[str, List[str]]):
        self.intent_samples = intent_samples
        # 全サンプルをフラットなリストに
        all_samples = [sample for samples in intent_samples.values() for sample in samples]
        
        # TF-IDF計算用のベクトライザを初期化
        self.char_vectorizer = TfidfVectorizer(
            analyzer='char',
            ngram_range=(1, 3)
        )
        # TF-IDFベクトルを事前計算
        self.tfidf_matrix = self.char_vectorizer.fit_transform(all_samples)
        
        # サンプルごとのインデックスを保持
        self.sample_indices = {}
        current_idx = 0
        for intent, samples in intent_samples.items():
            self.sample_indices[intent] = list(range(current_idx, current_idx + len(samples)))
            current_idx += len(samples)

    def normalize_text(self, text: str) -> str:
        """テキストの正規化"""
        text = jaconv.kata2hira(text.lower())
        text = ''.join(char for char in text if char.isalnum() or 'ぁ' <= char <= 'ん')
        return text

    @lru_cache(maxsize=1024)
    def text_to_phonemes(self, text: str) -> str:
        """音素列への変換"""
        try:
            _, phonemes = pyopenjtalk.g2p(text)
            phonemes = phonemes.replace(' ', '').replace('+', '').replace('*', '')
            return phonemes
        except:
            return self.normalize_text(text)

    def get_char_ngrams(self, text: str, n: int) -> List[str]:
        """文字n-gramの生成"""
        text = self.normalize_text(text)
        return [text[i:i+n] for i in range(len(text)-n+1)]

    def calculate_ngram_similarity(self, text1: str, text2: str) -> float:
        """n-gramベースの類似度計算"""
        # 1-gram, 2-gram, 3-gramの組み合わせで計算
        ngram_sims = []
        for n in [1, 2, 3]:
            ngrams1 = set(self.get_char_ngrams(text1, n))
            ngrams2 = set(self.get_char_ngrams(text2, n))
            if not ngrams1 or not ngrams2:
                continue
            # Jaccard類似度
            intersection = len(ngrams1 & ngrams2)
            union = len(ngrams1 | ngrams2)
            ngram_sims.append(intersection / union if union > 0 else 0)
        return np.mean(ngram_sims) if ngram_sims else 0.0

    def calculate_tfidf_similarity(self, text: str, sample_indices: List[int]) -> float:
        """TF-IDF類似度の計算"""
        text_vector = self.char_vectorizer.transform([text])
        similarities = []
        for idx in sample_indices:
            sample_vector = self.tfidf_matrix[idx]
            similarity = (text_vector @ sample_vector.T).toarray()[0][0]
            similarities.append(similarity)
        return max(similarities) if similarities else 0.0

    def calculate_similarity(self, text1: str, text2: str) -> Tuple[float, float, float, float]:
        """複数の類似度指標を計算"""
        # 編集距離ベースの類似度
        norm_text1 = self.normalize_text(text1)
        norm_text2 = self.normalize_text(text2)
        char_similarity = 1 - Levenshtein.distance(norm_text1, norm_text2) / max(len(norm_text1), len(norm_text2), 1)

        # 音素レベルの類似度
        phon_text1 = self.text_to_phonemes(text1)
        phon_text2 = self.text_to_phonemes(text2)
        phon_similarity = 1 - Levenshtein.distance(phon_text1, phon_text2) / max(len(phon_text1), len(phon_text2), 1)

        # n-gram類似度
        ngram_similarity = self.calculate_ngram_similarity(text1, text2)

        return char_similarity, phon_similarity, ngram_similarity

    # predictメソッドの閾値を調整
    def predict(self, text: str, 
        char_threshold: float = 0.7,    # 文字類似度の閾値を上げる
        phon_threshold: float = 0.7,    # 音素類似度の閾値を上げる
        ngram_threshold: float = 0.4,   # n-gram類似度の閾値を上げる
        tfidf_threshold: float = 0.3    # TF-IDF類似度の閾値を追加
    ) -> Tuple[str, Dict[str, float], Dict]:
        """複数の類似度指標を使用してintentを予測"""
        best_intent = "unknown"  # デフォルトをunknownに設定
        best_scores = {'char': 0.0, 'phon': 0.0, 'ngram': 0.0, 'tfidf': 0.0}
        
        results = {intent: {
            'scores': {'char': 0.0, 'phon': 0.0, 'ngram': 0.0, 'tfidf': 0.0},
            'matched_samples': [],
            'total_score': 0.0
        } for intent in self.intent_samples.keys()}

        # 重み付けを調整
        weights = {
            'char': 0.3,
            'phon': 0.3,
            'ngram': 0.2,
            'tfidf': 0.2
        }

        for intent, samples in self.intent_samples.items():
            # TF-IDF類似度の計算
            tfidf_sim = self.calculate_tfidf_similarity(text, self.sample_indices[intent])
            results[intent]['scores']['tfidf'] = tfidf_sim

            # その他の類似度の計算
            for sample in samples:
                char_sim, phon_sim, ngram_sim = self.calculate_similarity(text, sample)
                
                # 閾値判定を強化
                matches_threshold = (
                    (char_sim >= char_threshold) or
                    (phon_sim >= phon_threshold) or
                    (ngram_sim >= ngram_threshold) or
                    (tfidf_sim >= tfidf_threshold)
                )
                
                if matches_threshold:
                    results[intent]['matched_samples'].append({
                        'sample': sample,
                        'char_sim': char_sim,
                        'phon_sim': phon_sim,
                        'ngram_sim': ngram_sim,
                        'tfidf_sim': tfidf_sim
                    })

                # スコアの更新
                results[intent]['scores']['char'] = max(results[intent]['scores']['char'], char_sim)
                results[intent]['scores']['phon'] = max(results[intent]['scores']['phon'], phon_sim)
                results[intent]['scores']['ngram'] = max(results[intent]['scores']['ngram'], ngram_sim)

            # 総合スコアの計算
            total_score = sum(sim * weights[metric] 
                            for metric, sim in results[intent]['scores'].items())
            results[intent]['total_score'] = total_score

            # 最高スコアの更新（閾値を考慮）
            current_total_score = sum(results[intent]['scores'][m] * weights[m] for m in weights)
            if current_total_score > sum(best_scores[m] * weights[m] for m in weights):
                # スコアが十分高い場合のみintentを更新
                if current_total_score >= 0.4:  # 総合スコアの閾値
                    best_intent = intent
                    best_scores = results[intent]['scores'].copy()

        return best_intent, best_scores, results

intent_samples = {
    "confirm": [
        "はい",
        "予約完了",
        "はい、お願いします",
        "そうですね",
        "それで大丈夫です",
        "問題ありません",
        "確定でお願いします",
        "その内容で大丈夫です",
        "それでいいです",
        "その通りです",
        "その予約でお願いします",
    ],
    "cancel": [
        "いいえ",
        "やめておきます",
        "キャンセルします",
        "予約はしません",
        "キャンセルでお願いします",
        "取り消してください",
        "予約キャンセルで",
        "無しで",
    ],
    "change": [
        "時間を変更したいです",
        "人数を5名から3名に変更できますか",
        "日にちを変更したいです",
        "15時じゃなくて17時にしたいです",
        "人数を変更したいのですが",
        "時間を後ろにずらしたいです",
        "別の日にしたいです",
        "人数を増やしたいです",
        "もう少し遅い時間にできますか",
        "日付を来週にしたいです",
        "19時です",
        "3人です",
        "ちがいます 5人です",
        "やっぱり明後日で",
        "違う 5時"
    ]
}

classifier = TextSimilarityClassifier(intent_samples)

In [78]:
test_texts = [
    "はい",
    "やっぱりなしで",
    "人数を3名に変更できますか？",
    "やっぱり時間をもう少し遅くしたいです",
    "やっぱり5人で",
    "やっぱり17時で",
    "違う16時です",
    "ちがう4人です",
    "違います 10人です",
    "その予約で確定でお願いします",
    "やっぱりキャンセルします",
    "19時に変更できますか",
    "すみませんやっぱり",
    "すみませんやっぱり明後日で",
    "すみませんやっぱり5人で",
    "すみませんやっぱり二人で",
    "すみませんわかりません",  # unknown intentのテスト
]

for test_text in test_texts:
    intent, scores, results = classifier.predict(test_text)
    print(f"\nInput: {test_text}")
    print(f"Predicted Intent: {intent}")
    print("\nScores:")
    for metric, score in scores.items():
        print(f"{metric}: {score:.3f}")
    
    print("\nDetailed Results:")
    for intent, result in results.items():
        print(f"\n{intent}:")
        print("Scores:", {k: f"{v:.3f}" for k, v in result['scores'].items()})
        print(f"Total Score: {result['total_score']:.3f}")


Input: はい
Predicted Intent: confirm

Scores:
char: 1.000
phon: 1.000
ngram: 1.000
tfidf: 1.000

Detailed Results:

confirm:
Scores: {'char': '1.000', 'phon': '1.000', 'ngram': '1.000', 'tfidf': '1.000'}
Total Score: 1.000

cancel:
Scores: {'char': '0.333', 'phon': '0.333', 'ngram': '0.167', 'tfidf': '0.144'}
Total Score: 0.262

change:
Scores: {'char': '0.111', 'phon': '0.111', 'ngram': '0.056', 'tfidf': '0.037'}
Total Score: 0.085

Input: やっぱりなしで
Predicted Intent: change

Scores:
char: 0.625
phon: 0.625
ngram: 0.341
tfidf: 0.595

Detailed Results:

confirm:
Scores: {'char': '0.143', 'phon': '0.143', 'ngram': '0.061', 'tfidf': '0.066'}
Total Score: 0.111

cancel:
Scores: {'char': '0.286', 'phon': '0.286', 'ngram': '0.131', 'tfidf': '0.194'}
Total Score: 0.236

change:
Scores: {'char': '0.625', 'phon': '0.625', 'ngram': '0.341', 'tfidf': '0.595'}
Total Score: 0.562

Input: 人数を3名に変更できますか？
Predicted Intent: change

Scores:
char: 0.765
phon: 0.765
ngram: 0.681
tfidf: 0.812

Detailed Resul

In [81]:
# 既存のintent_samplesにunknownを追加
intent_samples = {
    "confirm": [
        "はい",
        "予約完了",
        "はい、お願いします",
        "そうですね",
        "それで大丈夫です",
        "問題ありません",
        "確定でお願いします",
        "その内容で大丈夫です",
        "それでいいです",
        "その通りです",
        "その予約でお願いします",
    ],
    "cancel": [
        "いいえ",
        "やめておきます",
        "キャンセルします",
        "予約はしません",
        "キャンセルでお願いします",
        "取り消してください",
        "予約キャンセルで",
        "無しで",
    ],
    "change": [
        "時間を変更したいです",
        "人数を5名から3名に変更できますか",
        "日にちを変更したいです",
        "15時じゃなくて17時にしたいです",
        "人数を変更したいのですが",
        "時間を後ろにずらしたいです",
        "別の日にしたいです",
        "人数を増やしたいです",
        "もう少し遅い時間にできますか",
        "日付を来週にしたいです",
        "やっぱり19時で",
        "やっぱり3人で",
        "ちがいます 5人です",
        "やっぱり明後日で",
        "違う7時です"
    ],
    "unknown": [
        "天気はどうですか",
        "お腹すいた",
        "眠い",
        "今何時ですか",
        "レストランの場所はどこですか",
        "メニューを見せてください",
        "支払い方法は何がありますか",
        "駐車場はありますか",
        "トイレはどこですか",
        "料金はいくらですか",
        "混んでいますか",
        "予約なしでも大丈夫ですか",
        "どんな料理がおいすすめですか",
        "営業時間は何時までですか",
    ]
}



# テスト用のコード
def test_classifier():
    classifier = TextSimilarityClassifier(intent_samples)
    
    test_texts = [
        "はい",
        "やっぱりなしで",
        "人数を3名に変更できますか？",
        "やっぱり時間をもう少し遅くしたいです",
        "やっぱり5人で",
        "やっぱり17時で",
        "違う16時です",
        "ちがう4人です",
        "違います 10人です",
        "その予約で確定でお願いします",
        "やっぱりキャンセルします",
        "19時に変更できますか",
        "すみませんやっぱり",
        "すみませんやっぱり明後日で",
        "すみませんやっぱり5人で",
        "すみませんやっぱり二人で",
        "すみませんわかりません",  # unknown intentのテスト
    ]

    for test_text in test_texts:
        intent, scores, results = classifier.predict(test_text)
        print(f"\nInput: {test_text}")
        print(f"Predicted Intent: {intent}")
        print("\nScores:")
        for metric, score in scores.items():
            print(f"{metric}: {score:.3f}")

test_classifier()


Input: はい
Predicted Intent: confirm

Scores:
char: 1.000
phon: 1.000
ngram: 1.000
tfidf: 1.000

Input: やっぱりなしで
Predicted Intent: change

Scores:
char: 0.714
phon: 0.714
ngram: 0.380
tfidf: 0.527

Input: 人数を3名に変更できますか？
Predicted Intent: change

Scores:
char: 0.765
phon: 0.765
ngram: 0.681
tfidf: 0.805

Input: やっぱり時間をもう少し遅くしたいです
Predicted Intent: change

Scores:
char: 0.444
phon: 0.444
ngram: 0.307
tfidf: 0.483

Input: やっぱり5人で
Predicted Intent: change

Scores:
char: 0.857
phon: 0.857
ngram: 0.500
tfidf: 0.637

Input: やっぱり17時で
Predicted Intent: change

Scores:
char: 0.875
phon: 0.875
ngram: 0.556
tfidf: 0.650

Input: 違う16時です
Predicted Intent: change

Scores:
char: 0.714
phon: 0.714
ngram: 0.375
tfidf: 0.632

Input: ちがう4人です
Predicted Intent: change

Scores:
char: 0.556
phon: 0.556
ngram: 0.288
tfidf: 0.495

Input: 違います 10人です
Predicted Intent: change

Scores:
char: 0.556
phon: 0.556
ngram: 0.318
tfidf: 0.594

Input: その予約で確定でお願いします
Predicted Intent: confirm

Scores:
char: 0.786
phon: 0.786
